In [1]:
import numpy as np
import pandas as pd
import math
from queue import *
from collections import Counter

In [2]:
# Read in data
with open('../pa2train.txt') as f:
    train = f.readlines()
train = [x.strip() for x in train] 

with open('../pa2validation.txt') as f:
    val = f.readlines()
val = [x.strip() for x in val] 

with open('../pa2test.txt') as f:
    test = f.readlines()
test = [x.strip() for x in test] 

In [3]:
# create empty lists for data sorting
trainData = []
trainLabel = []

valData = []
valLabel = []

testData = []
testLabel = []

In [4]:
# Remove spaces and split into label/data
for i in range (0,len(train)):
    train[i] = train[i].split(" ")
    trainLabel.append(train[i][22])
    trainData.append(train[i][:22])

for i in range (0,len(val)):
    val[i] = val[i].split(" ")
    valLabel.append(val[i][22])
    valData.append(val[i][:22])

for i in range (0,len(test)):
    test[i] = test[i].split(" ")
    testLabel.append(test[i][22])
    testData.append(test[i][:22])

In [5]:
# convert label string to int
trainLabel = [float(i) for i in trainLabel]

valLabel = [float(i) for i in valLabel]

testLabel = [float(i) for i in testLabel]

In [6]:
# convert data string to int
for i in range(0,len(trainData)):
    trainData[i] = [float(j) for j in trainData[i]]
    
for i in range(0,len(valData)):
    valData[i] = [float(j) for j in valData[i]]
    
for i in range(0,len(testData)):
    testData[i] = [float(j) for j in testData[i]]

In [7]:
len(trainData[0])

22

In [8]:
class Node:
    def __init__(self):
        self.left_child = None
        self.right_child = None
        self.rule = None
        self.labels = None
class Tree:
    def __init__(self):
        self.root = None

In [9]:
def calcLog(prob):
    if(prob == 0):
        return 0.0
    else:
        return (prob)*math.log(prob)

In [10]:
def splitData(data, labels):
    featEn = []

    # for i from 0 to number of features(# of columns)
    for i in range (0,len(data[0])):
        column = []
        threshold = []
    
        # for j from 0 to number of data points(# of rows)
        for j in range (0, len(data)):
            # getting each i feature value in the j column
            column.append(data[j][i])
        
        # get n unique values in the feature
        uFeat = list(set(column))

        uFeat.sort()
        
        if(len(uFeat) != 1):     
            # get thresholds between each unique value for n-1 threshold values
            for k in range (0, len(uFeat)-1):
                threshold.append((uFeat[k]+uFeat[k+1])/2)
            
            condEn = []
    
            # for m from 0 to number of thresholds(n-1)
            for m in range (0, len(threshold)):
                numLess = 0
                lessYes = 0
                lessNo = 0
                greatYes = 0
                greatNo = 0
        
                # count number of features less than the threshold (P(feature) = [probLess,probGreat])
                for n in range (0, len(column)):
                    if(column[n] < threshold[m]):
                        if(labels[n] == 1):
                            lessYes += 1
                        else:
                            lessNo += 1
                        numLess += 1
                    else:
                        if(labels[n] == 1):
                            greatYes += 1
                        else:
                            greatNo += 1
                
                # (P(feature) = [probLess,probGreat])               
                probLess = numLess/len(column)
                probGreat = (len(column)-numLess)/len(column)
    
                # P(label|feature < threshold)
                less = (lessYes/numLess,lessNo/numLess)
                # P(label|feature >= threshold)
                great = (greatYes/(len(column)-numLess),greatNo/(len(column)-numLess))
    
                # calculate cond entropy
                en = probLess*(-(calcLog(less[0]) + calcLog(less[1]))) + probGreat*(-(calcLog(great[0]) + calcLog(great[1])))
    
                # append threshold value and its entropy to list
                condEn.append((threshold[m], en))
        
       
            # sort entropy for that feature
            condEn.sort(key=lambda tup: tup[1]) 

            # put the lowest entropy of each feat in a list
            featEn.append((condEn[0][0],condEn[0][1],i))
            
    featEn.sort()
    
    return featEn[0]

In [62]:
out

[(0.5, 0.3299390039689053, 4),
 (0.5, 0.5007440291663044, 5),
 (0.5, 0.5912692880769884, 6),
 (0.5, 0.614920196492256, 7),
 (0.5, 0.674032026694068, 2),
 (1.0, 0.6275685664718434, 9),
 (1.0, 0.6300485281912376, 8),
 (1.5, 0.6711446686784288, 1),
 (1.5, 0.6739912420558656, 3),
 (9.5, 0.6666625790814404, 18),
 (23.0, 0.6704963092901864, 19),
 (28.5, 0.6697377831744773, 21),
 (31.5, 0.6636594641359148, 10),
 (33.5, 0.6718070296274592, 20),
 (56.0, 0.6724794152719034, 11),
 (67.5, 0.6533529585902823, 17),
 (75.0, 0.6236341711936715, 16),
 (25000.0, 0.6603981245909911, 0),
 (28569.0, 0.6710047933781556, 12),
 (29817.0, 0.6677503803145969, 13),
 (30461.5, 0.6676054720354984, 15),
 (33539.5, 0.6668831061644981, 14)]

In [11]:
def setSplit(data,rule,labels):
    # left and right data + label
    left = []
    leftLabel = []
    right = []
    rightLabel = []

    # for each i data point
    for i in range (0,len(data)):
        # check if the data point in the feature is < lowest entropy threshold chosen
        if(data[i][rule[2]] < rule[0]):
            left.append(data[i])
            leftLabel.append(labels[i])
        else:
            right.append(data[i])
            rightLabel.append(labels[i])
    # return left data + label and right data + label
    return left, right, leftLabel, rightLabel       

In [19]:
(left,right,leftLabel,rightLabel) = setSplit(trainData,out,trainLabel)

In [12]:
def createNode(data,labels):
    if(len(set(labels)) == 1):
        return (data,labels)
    
    curNode = Node()
    
    # decide where to split
    bestSplit = splitData(data,labels)
    (left,right,leftLabel,rightLabel) = setSplit(data,bestSplit,labels)
    
    # assign split data to children
    curNode.left_child = createNode(left,leftLabel)
    curNode.right_child = createNode(right,rightLabel)
    curNode.rule = bestSplit
    
    return curNode

In [13]:
# create new tree
id3Tree = Tree()
# create root node for the tree
rootNode = Node()
# give it the training data
rootNode.rule = trainData
rootNode.labels = trainLabel

# assign root to tree
id3Tree.root = rootNode

# create queue for iteration and add root node
nodeQueue = Queue()
nodeQueue.put(rootNode)

In [14]:
while(nodeQueue.qsize() != 0):
    # get out the top node in queue
    curNode = nodeQueue.get()
    
    # if node is not pure:
    if(len(set(curNode.labels)) != 1):
        # decide where to split
        bestSplit = splitData(curNode.rule,curNode.labels)
    
        # split data via the new rule
        (left,right,leftLabel,rightLabel) = setSplit(curNode.rule,bestSplit,curNode.labels)
       
        # create node children
        leftNode = Node()
        rightNode = Node()
    
        # sign data/labels to children
        leftNode.rule = left
        leftNode.labels = leftLabel
        rightNode.rule = right
        rightNode.labels = rightLabel
    
        # change rule to splitting rule instead of data
        curNode.rule = bestSplit
        # change labels to count of how many data points were split in the node
        curNode.labels = len(curNode.labels)
        
        # assign children to current node
        curNode.left_child = leftNode
        curNode.right_child = rightNode  
    
        # put new nodes in queue if len > 1
        if(len(leftNode.rule) > 1):
            nodeQueue.put(leftNode)
        if(len(rightNode.rule) > 1):
            nodeQueue.put(rightNode)

In [24]:
set(id3Tree.root.left_child.left_child.labels)

TypeError: 'set' object does not support indexing

In [20]:
rootNode.rule

(0.5, 0.3299390039689053, 4)

In [27]:
# init prediction
trainPred = []

# predict training data
for i in range (0,len(trainData)):
    curNode = id3Tree.root
    
    while(curNode.right_child != None and curNode.left_child != None):
        if(trainData[i][curNode.rule[2]] < curNode.rule[0]):
            curNode = curNode.left_child
        else:
            curNode = curNode.right_child
            
    nodeLabels = curNode.labels
    # if there is only one label type in labels
    if(len(list(set(nodeLabels))) == 1):
        trainPred.append(nodeLabels[0])
    # if there is more than one label predict majority
    else:
        # count number of 1 or 0
        countOne = nodeLabels.count(1)
        countZero = nodeLabels.count(0)
        # append correct label predicted
        if(countOne > countZero):
            trainPred.append(1)
        else:
            trainPred.append(0)
    

In [33]:
# caluclate training error
counter = 0

for i in range (0, len(trainPred)):
    if(trainPred[i] == trainLabel[i]):
        counter += 1

trainErr = (len(trainPred)-counter)/len(trainPred)

In [34]:
trainErr

0.0